In [1]:
import subprocess
import pandas as pd
import numpy as np
import random
from helper_fns import *

random.seed(42)

In [2]:
EUR_ge_regressed, YRI_ge_regressed, EUR_protein_genes, YRI_protein_genes = load_data()

Shapes of the dataframes: (373, 13942) (89, 13942) (13942, 5) (13942, 5)


In [5]:
EUR_protein_genes

,start,end,gene_id,name,chr
0,360260,1360261,ENSG00000187634,SAMD11,1
1,379584,1379585,ENSG00000188976,NOC2L,1
2,395967,1395968,ENSG00000187961,KLHL17,1
3,401877,1401878,ENSG00000187583,PLEKHN1,1
4,410579,1410580,ENSG00000187642,PERM1,1
...,...,...,...,...,...
13937,139949356,140949357,ENSG00000148399,DPH7,9
13938,139976531,140976532,ENSG00000165724,ZMYND19,9
13939,140000106,141000107,ENSG00000197070,ARRDC1,9
13940,140013444,141013445,ENSG00000181090,EHMT1,9


In [6]:
# Function to extract SNPs for a gene and split into train/test
def process_gene(gene, chr, start, end, plink_path='plink'):
    base_name = f'{gene}_chr{chr}_{start}_{end}'
    # Step 2: Extract gene-specific SNPs
    subprocess.run([plink_path, '--bfile', f'chr{chr}', '--chr', str(chr), 
                    '--from-bp', str(start), '--to-bp', str(end), 
                    '--make-bed', '--out', base_name])
    
    # Count the number of individuals
    with open(f'{base_name}.fam') as f:
        num_individuals = sum(1 for line in f)
    
    # Calculate split
    train_num = int(num_individuals * 0.8)
    
    # Shuffle individuals
    individuals = pd.read_csv(f'{base_name}.fam', sep='\s+', header=None)
    shuffled = individuals.sample(frac=1).reset_index(drop=True)
    
    # Split into train/test
    train = shuffled.head(train_num)
    test = shuffled.tail(num_individuals - train_num)
    
    train[[0, 1]].to_csv(f'{base_name}_train.txt', sep=' ', index=False, header=False)
    test[[0, 1]].to_csv(f'{base_name}_test.txt', sep=' ', index=False, header=False)
    
    # Generate train/test datasets
    subprocess.run([plink_path, '--bfile', base_name, '--keep', f'{base_name}_train.txt', 
                    '--make-bed', '--out', f'{base_name}_train'])
    subprocess.run([plink_path, '--bfile', base_name, '--keep', f'{base_name}_test.txt', 
                    '--make-bed', '--out', f'{base_name}_test'])

In [7]:
os.chdir("/Users/xbh0403/Desktop/TWAS_ASSOC/project_data/geno_gene_specific/")

In [ ]:
process_gene('ENSG00000187634', 1, 360260, 1360261)